# Line detection

In [1]:
import json
import collections
import numpy as np
import sys
from datetime import datetime
from kafka import KafkaConsumer
   
def get_line_weights(current_players, lines, goalkeeper):
    line_weights = []
    current_players = np.array(current_players)
    current_players = current_players[current_players != goalkeeper]
    for line in lines:
        line_weights.append(np.sum([1 for player in current_players if player in line])/len(current_players))
    return line_weights

last_players_team_0 = []
last_players_team_1 = []
last_line_team_0 = -1
last_line_team_1 = -1
last_line_time_team_0 = None
last_line_time_team_1 = None
t0_goalkeeper = 30
t0_line1 = [6, 11, 12, 24, 99]
t0_line2 = [8, 14, 19, 21, 23]
t0_line3 = [2, 5, 9, 17, 37]
t0_lines = [t0_line1, t0_line2, t0_line3]
t0_last_line_chg = None
t0_current_line_time = None
t1_goalkeeper = 67
t1_line1 = [3, 8, 15, 16, 68]
t1_line2 = [13, 14, 28, 81, 90]
t1_line3 = [4, 11, 18, 24, 73]
t1_lines = [t1_line1, t1_line2, t1_line3]
t1_last_line_chg = None
t1_current_line_time = None
line_names = ["Line 1", "Line 2", "Line 3"]
current_timestamp = None

group_id = "line-detection-3"
consumer = KafkaConsumer(bootstrap_servers='86.119.35.55', 
                         auto_offset_reset='earliest', 
                         group_id=group_id)
consumer.subscribe('playerPositionGroupedByTime')

print("Start polling...")
while True:
    #try:    
        
        """ Kafka read message """
        records  = consumer.poll(10.0, update_offsets=True)
       
        for topic_data, consumer_records in records.items():
            for consumer_record in consumer_records:
                msg_json = json.loads(consumer_record.value.decode('utf-8'))

                eventExists = False
                current_players_team_0 = [int(x) for x in list(msg_json["teams"]["team_0"].keys())]
                current_players_team_1 = [int(x) for x in list(msg_json["teams"]["team_1"].keys())]

                """ Preprocessing """
               # Filter out junk data
                if msg_json["time"] == 0 or len(current_players_team_0) < 6 or len(current_players_team_1) < 6:
                    continue
                # Initialize line change timestamp
                current_timestamp = datetime.strptime(msg_json["time"], '%Y-%m-%d %H:%M:%S.%f%z')
                if t0_last_line_chg == None or t1_last_line_chg == None:
                    t0_last_line_chg = current_timestamp
                    t1_last_line_chg = current_timestamp
                
                """ Player changed """
                # New player 
                for player in current_players_team_0:
                    if player not in last_players_team_0:
                        #print(current_timestamp, "Player", player, " has joined Team 0", consumer_record.offset)
                        eventExists = True
                for player in current_players_team_1:
                    if player not in last_players_team_1:
                        #print(current_timestamp, "Player", player, " has joined Team 1", consumer_record.offset)    
                        eventExists = True
                # Player has left the field
                for player in last_players_team_0:
                    if player not in current_players_team_0:
                        #print(current_timestamp, "Player", player, " has left Team 0", consumer_record.offset)
                        eventExists = True
                for player in last_players_team_1:
                    if player not in current_players_team_1:
                        #print(current_timestamp, "Player", player, " has left Team 1", consumer_record.offset)
                        eventExists = True

                """ Line detection """
                # Line detection
                current_line_weights_team_0 = get_line_weights(current_players=current_players_team_0, lines=t0_lines, goalkeeper=t0_goalkeeper)
                current_line_weights_team_1 = get_line_weights(current_players=current_players_team_1, lines=t1_lines, goalkeeper=t1_goalkeeper)

                current_line_team_0 = np.argmax(current_line_weights_team_0)
                current_line_team_1 = np.argmax(current_line_weights_team_1)
                
                t0_current_line_time = (current_timestamp - t0_last_line_chg).seconds
                t1_current_line_time = (current_timestamp - t1_last_line_chg).seconds
                
                # Line change team 0
                if current_line_team_0 != last_line_team_0:
                    print(current_timestamp, "Team 0 has switched from", line_names[last_line_team_0], "to", line_names[current_line_team_0], consumer_record.offset)
                    t0_last_line_chg = current_timestamp
                    eventExists = True
                # Line change team 1
                if current_line_team_1 != last_line_team_1:
                    print(current_timestamp, "Team 1 has switched from", line_names[last_line_team_1], "to", line_names[current_line_team_1], consumer_record.offset)
                    t1_last_line_chg = current_timestamp
                    eventExists = True
                    
                if last_line_time_team_0 != t0_current_line_time or last_line_time_team_1 != t1_current_line_time:
                    eventExists = True
                                
                if eventExists:
                    print("Players Team 0:", sorted(current_players_team_0), ", line time:", t0_current_line_time)
                    print("Players Team 1:", sorted(current_players_team_1), ", line time:", t1_current_line_time)

                # Set variables for next iteration
                last_players_team_0 = current_players_team_0
                last_players_team_1 = current_players_team_1
                last_line_team_0 = current_line_team_0
                last_line_team_1 = current_line_team_1
                last_line_time_team_0 = t0_current_line_time
                last_line_time_team_1 = t1_current_line_time                
        
            continue
    #except Exception as e:
        #print(e)
        #consumer.close()
        #break
    #finally:
        #consumer.close()

Start polling...
2022-08-21 10:30:45.100000+01:00 Team 0 has switched from Line 3 to Line 1 386328
2022-08-21 10:30:45.100000+01:00 Team 1 has switched from Line 3 to Line 1 386328
Players Team 0: [6, 11, 12, 24, 30, 99] , line time: 0
Players Team 1: [3, 8, 15, 16, 67, 68] , line time: 0
Players Team 0: [6, 11, 12, 24, 30, 99] , line time: 1
Players Team 1: [3, 8, 15, 16, 67, 68] , line time: 1
Players Team 0: [6, 11, 12, 24, 30, 99] , line time: 2
Players Team 1: [3, 8, 15, 16, 67, 68] , line time: 2
Players Team 0: [6, 11, 12, 24, 30, 99] , line time: 3
Players Team 1: [3, 8, 15, 16, 67, 68] , line time: 3
Players Team 0: [6, 11, 12, 24, 30, 99] , line time: 4
Players Team 1: [3, 8, 15, 16, 67, 68] , line time: 4
Players Team 0: [6, 11, 12, 24, 30, 99] , line time: 5
Players Team 1: [3, 8, 15, 16, 67, 68] , line time: 5
Players Team 0: [6, 11, 12, 24, 30, 99] , line time: 6
Players Team 1: [3, 8, 15, 16, 67, 68] , line time: 6
Players Team 0: [6, 11, 12, 24, 30, 99] , line time: 7
P

KeyboardInterrupt: 

In [9]:
str(current_timestamp)

response_msg= {}
response_msg["time"] = str(current_timestamp)

print(response_msg)

response_msg= {}
response_team_0 = {}
response_team_1 = {}
response_msg["time"] = str(current_timestamp)
response_team_0["players"] = sorted(current_players_team_0)
response_team_1["players"] = sorted(current_players_team_1)
response_msg["team_0"] = response_team_0
response_msg["team_1"] = response_team_1

print(json.dumps(response_msg))


{'time': '2022-08-21 10:31:07.934000+01:00'}
{"time": "2022-08-21 10:31:07.934000+01:00", "team_0": {"players": [6, 11, 12, 24, 30, 99]}, "team_1": {"players": [3, 8, 15, 16, 67, 68]}}
